# 컴피티션 개인전
- 17시30분 마감
- 평가지표: AUC
- 1~3등에게 스타벅스 쿠폰을 전달

In [22]:
%pip install catboost

In [23]:
%pip install optuna

In [24]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import KFold, cross_val_score
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
# from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import optuna

In [25]:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 데이터 경로와 시드값

In [26]:
DATA_PATH  = "/content/drive/MyDrive/the_datas/data/"
SEED = 42

# 데이터 불러오기

In [27]:
train_tr = pd.read_csv(f"{DATA_PATH}store_train_transactions.csv") # 학습용 구매기록 데이터
train_target = pd.read_csv(f"{DATA_PATH}store_train.csv") # 학습용 정답 데이터
test_tr = pd.read_csv(f"{DATA_PATH}store_test_transactions.csv") # 테스트용 구매기록 데이터
submit = pd.read_csv(f"{DATA_PATH}store_submission.csv") # 제출 양식 데이터
train_tr.shape ,test_tr.shape, train_target.shape, submit.shape

((523105, 7), (441196, 7), (14940, 2), (12225, 2))

In [28]:
train_ft = pd.read_csv(f"{DATA_PATH}train_common_4.csv") # 학습 데이터(피처)
test_ft = pd.read_csv(f"{DATA_PATH}test_common_4.csv") # 테스트 데이터(피처)

train_ft.shape , test_ft.shape

((14940, 450), (12225, 450))

In [29]:
# display(train.head())
# display(train.info())

- 공통 피처 파일 불러오기

In [30]:
# train_ft = pd.read_csv(f"{DATA_PATH}train_common_3.csv") # 학습 데이터(피처)
# test_ft = pd.read_csv(f"{DATA_PATH}test_common_3.csv") # 테스트 데이터(피처)

# train_ft.shape , test_ft.shape

# 결측치 처리

In [31]:
mask = train_ft.isnull().sum() > 0
train_ft.isnull().sum()[mask]

,0
3회이상count,4214
구매금액표준편차,388
구매금액왜도,840
구매금액첨도,1265


In [32]:
mask = test_ft.isnull().sum() > 0
test_ft.isnull().sum()[mask]

,0
3회이상count,3284
구매금액표준편차,242
구매금액왜도,599
구매금액첨도,954


In [33]:
train_ft["3회이상count"] = train_ft["3회이상count"].fillna(0)
train_ft["구매금액표준편차"] = train_ft["구매금액표준편차"].fillna(0)
train_ft["구매금액왜도"] = train_ft["구매금액왜도"].fillna(0)
train_ft["구매금액첨도"] = train_ft["구매금액첨도"].fillna(0)

test_ft["3회이상count"] = test_ft["3회이상count"].fillna(0)
test_ft["구매금액표준편차"] = test_ft["구매금액표준편차"].fillna(0)
test_ft["구매금액왜도"] = test_ft["구매금액왜도"].fillna(0)
test_ft["구매금액첨도"] = test_ft["구매금액첨도"].fillna(0)

In [34]:
train_ft.isnull().sum().sum(), test_ft.isnull().sum().sum()

(0, 0)

# 특성 공학(Feature Engineering)

- ID 변수 제외

In [35]:
train_ft = train_ft.iloc[:,1:]
test_ft = test_ft.iloc[:,1:]
train_ft.shape, test_ft.shape

((14940, 449), (12225, 449))

- 추가 피처 만들어 보기

In [36]:
train_ft

,구매횟수,내점일수,구매주기,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,12시이전구매비율,12시이후_18시이전구매비율,...,대_pivot_횟수비율A373000_공산품,대_pivot_횟수비율A373000_남성의류,대_pivot_횟수비율A373000_명품잡화,대_pivot_횟수비율A373000_생식품,대_pivot_횟수비율A373000_아동_스포츠,대_pivot_횟수비율A373000_여성정장,대_pivot_횟수비율A373000_여성캐주얼,대_pivot_횟수비율A373000_패션잡화,하루 구매 시간 간격_y,대분류_영_cnt
0,16,11,23,0.050000,0.250000,0.400000,0.300000,3,0.300000,0.450000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2220.0,0
1,41,21,16,0.357143,0.166667,0.357143,0.119048,3,0.214286,0.476190,...,0.048780,0.000000,0.048780,0.000000,0.000000,0.024390,0.048780,0.000000,14580.0,15
2,102,56,6,0.464912,0.140351,0.175439,0.219298,0,0.122807,0.728070,...,0.157895,0.157895,1.736842,0.157895,0.947368,0.157895,0.263158,0.368421,16680.0,0
3,191,92,3,0.379147,0.180095,0.236967,0.203791,3,0.161137,0.616114,...,0.000000,0.100000,0.200000,0.000000,0.000000,0.050000,0.300000,0.000000,25800.0,9
4,55,27,11,0.112903,0.612903,0.209677,0.064516,4,0.096774,0.741935,...,0.047619,0.000000,0.714286,0.000000,0.000000,0.000000,0.000000,0.000000,18000.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14935,16,10,24,0.000000,0.578947,0.052632,0.368421,2,0.263158,0.578947,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3600.0,3
14936,8,8,29,0.125000,0.625000,0.250000,0.000000,0,0.000000,0.500000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0
14937,22,12,26,0.208333,0.458333,0.208333,0.125000,1,0.375000,0.541667,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3600.0,3
14938,31,12,28,0.393939,0.393939,0.212121,0.000000,3,0.151515,0.484848,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9180.0,24


In [37]:
cols = [ col for col in train_ft.columns if col.startswith("중_pivot_횟수비율_") ]

In [38]:
# cols

In [39]:
# train_ft["중분류별_구매횟수_std"] = train_ft[cols].std(axis=1)
# train_ft["중분류별_구매횟수_skew"] = train_ft[cols].skew(axis=1)
# train_ft["중분류별_구매횟수_kurt"] = train_ft[cols].kurt(axis=1)

# test_ft["중분류별_구매횟수_std"] = test_ft[cols].std(axis=1)
# test_ft["중분류별_구매횟수_skew"] = test_ft[cols].skew(axis=1)
# test_ft["중분류별_구매횟수_kurt"] = test_ft[cols].kurt(axis=1)

# train_ft.shape, test_ft.shape

In [40]:
# 최소 구매금액범주화
# 중분류 개수

## Feature Encoding

In [41]:
cols = train_ft.select_dtypes("object").columns.tolist()
train_ft[cols].nunique()

,0
주구매지점,4
최대구매액_대분류,28
최소구매액_대분류,28


In [42]:
%pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 5.9 MB/s eta 0:00:00


In [43]:
import category_encoders as ce

In [44]:
enc = ce.one_hot.OneHotEncoder()
tmp = enc.fit_transform(train_ft[["주구매지점"]])
train_ft = pd.concat([train_ft,tmp],axis =1)

tmp = enc.transform(test_ft[["주구매지점"]])
test_ft = pd.concat([test_ft,tmp],axis =1)

train_ft.shape, test_ft.shape

((14940, 453), (12225, 453))

In [45]:
enc = ce.one_hot.OneHotEncoder()
tmp = enc.fit_transform(train_ft[["최소구매액_대분류"]])
train_ft = pd.concat([train_ft,tmp],axis =1)

tmp = enc.transform(test_ft[["최소구매액_대분류"]])
test_ft = pd.concat([test_ft,tmp],axis =1)

train_ft.shape, test_ft.shape

((14940, 481), (12225, 481))

In [46]:
enc = ce.one_hot.OneHotEncoder()
tmp = enc.fit_transform(train_ft[["최대구매액_대분류"]])
train_ft = pd.concat([train_ft,tmp],axis =1)

tmp = enc.transform(test_ft[["최대구매액_대분류"]])
test_ft = pd.concat([test_ft,tmp],axis =1)

train_ft.shape, test_ft.shape

((14940, 509), (12225, 509))

In [47]:
# enc = ce.count.CountEncoder()
# train_ft["최소구매액_대분류_cnt"] = enc.fit_transform(train_ft[["최소구매액_대분류"]])
# test_ft["최소구매액_대분류_cnt"] = enc.transform(test_ft[["최소구매액_대분류"]])

# train_ft.shape, test_ft.shape

In [48]:
# enc = ce.count.CountEncoder()
# train_ft["최대구매액_대분류_cnt"] = enc.fit_transform(train_ft[["최대구매액_대분류"]])
# test_ft["최대구매액_대분류_cnt"] = enc.transform(test_ft[["최대구매액_대분류"]])

# train_ft.shape, test_ft.shape

- 문자열 피처 삭제

In [49]:
cols

['주구매지점', '최대구매액_대분류', '최소구매액_대분류']

In [50]:
train_ft = train_ft.drop(columns=cols)
test_ft = test_ft.drop(columns=cols)
train_ft.shape, test_ft.shape

((14940, 506), (12225, 506))

In [51]:
train_ft.select_dtypes("object").columns , test_ft.select_dtypes("object").columns

(Index([], dtype='object'), Index([], dtype='object'))

## Feature Scaling

In [52]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [53]:
train_ft[train_ft.columns] = scaler.fit_transform(train_ft)
test_ft[test_ft.columns] = scaler.transform(test_ft)
train_ft.head()

,구매횟수,내점일수,구매주기,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,12시이전구매비율,12시이후_18시이전구매비율,...,최대구매액_대분류_19,최대구매액_대분류_20,최대구매액_대분류_21,최대구매액_대분류_22,최대구매액_대분류_23,최대구매액_대분류_24,최대구매액_대분류_25,최대구매액_대분류_26,최대구매액_대분류_27,최대구매액_대분류_28
0,-0.437126,-0.369867,0.002987,-1.029777,0.001191,0.838272,0.338186,0.109631,0.718557,-0.654150,...,-0.193649,-0.131248,-0.07785,-0.1402,-0.200973,-0.090737,-0.071977,-0.146044,-0.067118,-0.050497
1,0.239394,0.144110,-0.356452,0.323951,-0.390607,0.620171,-0.552996,0.109631,0.269465,-0.542415,...,-0.193649,-0.131248,-0.07785,-0.1402,-0.200973,-0.090737,-0.071977,-0.146044,-0.067118,-0.050497
2,1.890101,1.943028,-0.869935,0.798943,-0.514333,-0.304527,-0.059266,-1.643370,-0.209830,0.532172,...,-0.193649,-0.131248,-0.07785,-0.1402,-0.200973,-0.090737,-0.071977,-0.146044,-0.067118,-0.050497
3,4.298510,3.793345,-1.023980,0.420933,-0.327474,0.008592,-0.135636,0.109631,-0.009001,0.054536,...,-0.193649,-0.131248,-0.07785,-0.1402,-0.200973,-0.090737,-0.071977,-0.146044,-0.067118,-0.050497
4,0.618244,0.452496,-0.613193,-0.752532,1.707410,-0.130285,-0.821561,0.693965,-0.346226,0.591326,...,-0.193649,-0.131248,-0.07785,-0.1402,-0.200973,-0.090737,-0.071977,-0.146044,-0.067118,-0.050497


# 정답 데이터

In [54]:
target = train_target["target"]
target

,target
0,1.0
1,1.0
2,0.0
3,0.0
4,0.0
...,...
14935,0.0
14936,0.0
14937,0.0
14938,1.0


# 모델 튜닝하기

In [55]:
class RF_Objective:
    def __init__(self, x, y, seed):
        self.x = x
        self.y = y
        self.seed = seed
        self.cv = KFold(n_splits=5, shuffle=True, random_state=self.seed)

    def __call__(self, trial):
        hp = {
              'n_estimators': trial.suggest_int('n_estimators', 100, 1001),
              'criterion': trial.suggest_categorical('criterion', ['gini', 'entropy']),
              'max_depth': trial.suggest_int('max_depth', 3, 41),
              'min_samples_split': trial.suggest_int('min_samples_split', 6, 30),
              'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None])
        }
        model = RandomForestClassifier(random_state=self.seed, **hp)
        scores = cross_val_score(model, self.x, self.y, cv=self.cv, scoring='f1_macro', n_jobs=-1)
        return scores.mean()

class XG_Objective:
    def __init__(self, x, y, seed):
        self.x = x
        self.y = y
        self.seed = seed
        self.cv = KFold(n_splits=5, shuffle=True, random_state=self.seed)

    def __call__(self, trial):
        hp = {
            'n_estimators': trial.suggest_int('n_estimators', 50,201),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.31, step=0.01),
            'max_depth': trial.suggest_int('max_depth', 3, 12),
            'gamma': trial.suggest_float('gamma', 0.1, 5.1, step=0.1),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0, step=0.1),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0, step=0.1)
            }
        model = XGBClassifier(random_state=self.seed, **hp)
        scores = cross_val_score(model, self.x, self.y, cv=self.cv, scoring='f1_macro', n_jobs=-1)
        return scores.mean()

class LR_Objective:
    def __init__(self, x, y, seed):
        self.x = x
        self.y = y
        self.seed = seed
        self.cv = KFold(n_splits=5, shuffle=True, random_state=self.seed)

    def __call__(self, trial):
        hp = {
            'C': trial.suggest_float('C', 0.001, 10, step=0.001),
            'penalty': trial.suggest_categorical('penalty', ['l2',  None]),
            }
        model = LogisticRegression(random_state=self.seed, **hp)
        scores = cross_val_score(model, self.x, self.y, cv=self.cv, scoring='f1_macro', n_jobs=-1)
        return scores.mean()

class LGB_Objective:
    def __init__(self, x, y, seed):
        self.x = x
        self.y = y
        self.seed = seed
        self.cv = KFold(n_splits=5, shuffle=True, random_state=self.seed)

    def __call__(self, trial):
        hp = {
            'n_estimators': trial.suggest_int('n_estimators', 50, 501),
            'learning_rate': trial.suggest_float('learning_rate',0.01, 0.31, step=0.01),
            'max_depth': trial.suggest_int('max_depth', 3, 16),
            'num_leaves': trial.suggest_int('num_leaves', 20,101),
            'min_child_samples': trial.suggest_int('min_child_samples', 5,31),
            }
        model = LGBMClassifier(random_state=self.seed, **hp)
        scores = cross_val_score(model, self.x, self.y, cv=self.cv, scoring='f1_macro', n_jobs=-1)
        return scores.mean()

In [56]:
sampler = optuna.samplers.TPESampler(seed=SEED) # 대체 모델

In [57]:
rf_study = optuna.create_study(
    direction='maximize',
    sampler= sampler
)

[I 2024-11-13 13:17:39,687] A new study created in memory with name: no-name-78784062-9fcf-43c0-9c8d-7ff64c7d1a5b


In [58]:
rf_objective_func = RF_Objective(train_ft, target, SEED)
rf_study.optimize(rf_objective_func, n_trials=10)

[W 2024-11-13 13:18:06,869] Trial 0 failed with parameters: {'n_estimators': 437, 'criterion': 'gini', 'max_depth': 26, 'min_samples_split': 9, 'max_features': None} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-55-ad2f7da7b6a5>", line 17, in __call__
    scores = cross_val_score(model, self.x, self.y, cv=self.cv, scoring='f1_macro', n_jobs=-1)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/_param_validation.py", line 213, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 712, in cross_val_score
    cv_results = cross_validate(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/_param_validation.py", line 213, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/py

KeyboardInterrupt: 

In [ ]:
rf_study.best_value, rf_study.best_trial.params

In [ ]:
lr_study = optuna.create_study(
    direction='maximize',
    sampler= sampler
)

[I 2024-11-12 03:39:51,029] A new study created in memory with name: no-name-2a7d4960-6c12-4834-9527-504f0f1aa33c


In [ ]:
mask = train_ft.isna().sum() > 0
train_ft.isna().sum()[mask]

,0


In [ ]:
lr_objective_func = LR_Objective(train_ft, target, SEED)
lr_study.optimize(lr_objective_func, n_trials=200)

[I 2024-11-12 03:40:04,053] Trial 0 finished with value: 0.690972824354945 and parameters: {'C': 3.746, 'penalty': 'l2'}. Best is trial 0 with value: 0.690972824354945.
[I 2024-11-12 03:40:11,672] Trial 1 finished with value: 0.6911569197568891 and parameters: {'C': 5.987, 'penalty': 'l2'}. Best is trial 1 with value: 0.6911569197568891.
[I 2024-11-12 03:40:17,337] Trial 2 finished with value: 0.6914464002374353 and parameters: {'C': 0.581, 'penalty': 'l2'}. Best is trial 2 with value: 0.6914464002374353.
[I 2024-11-12 03:40:25,852] Trial 3 finished with value: 0.6914750490825103 and parameters: {'C': 7.081, 'penalty': None}. Best is trial 3 with value: 0.6914750490825103.
[I 2024-11-12 03:40:31,871] Trial 4 finished with value: 0.6913236145783025 and parameters: {'C': 8.325, 'penalty': 'l2'}. Best is trial 3 with value: 0.6914750490825103.
[I 2024-11-12 03:40:39,971] Trial 5 finished with value: 0.6914750490825103 and parameters: {'C': 1.835, 'penalty': None}. Best is trial 3 with val

In [ ]:
lr_study.best_value, lr_study.best_trial.params

(0.6915735318924592, {'C': 5.266, 'penalty': 'l2'})

In [59]:
xg_study = optuna.create_study(
    direction='maximize',
    sampler= sampler
)

[I 2024-11-13 13:18:12,281] A new study created in memory with name: no-name-bceaab89-53dc-4852-be51-1c45fd377f07


In [60]:

xg_objective_func = XG_Objective(train_ft, target, SEED)
xg_study.optimize(xg_objective_func, n_trials=150)

[I 2024-11-13 13:19:08,489] Trial 0 finished with value: 0.7143353834583229 and parameters: {'n_estimators': 141, 'learning_rate': 0.22, 'max_depth': 3, 'gamma': 5.0, 'subsample': 0.9, 'colsample_bytree': 0.6}. Best is trial 0 with value: 0.7143353834583229.
[I 2024-11-13 13:19:42,437] Trial 1 finished with value: 0.7100426130466844 and parameters: {'n_estimators': 77, 'learning_rate': 0.060000000000000005, 'max_depth': 6, 'gamma': 2.7, 'subsample': 0.7, 'colsample_bytree': 0.6}. Best is trial 0 with value: 0.7143353834583229.
[I 2024-11-13 13:20:29,170] Trial 2 finished with value: 0.7113947089566295 and parameters: {'n_estimators': 143, 'learning_rate': 0.05, 'max_depth': 5, 'gamma': 1.9000000000000001, 'subsample': 0.7, 'colsample_bytree': 0.9}. Best is trial 0 with value: 0.7143353834583229.
[I 2024-11-13 13:21:18,090] Trial 3 finished with value: 0.7045506167254144 and parameters: {'n_estimators': 80, 'learning_rate': 0.16, 'max_depth': 8, 'gamma': 0.30000000000000004, 'subsample'

In [61]:
xg_study.best_value, xg_study.best_trial.params

(0.7186924154921313,
 {'n_estimators': 138,
  'learning_rate': 0.13,
  'max_depth': 5,
  'gamma': 4.2,
  'subsample': 0.8,
  'colsample_bytree': 0.8})

In [62]:
# xg_study.best_trial.params =

In [63]:
lgb_study = optuna.create_study(
    direction='maximize',
    sampler= sampler
)

lgb_objective_func = LGB_Objective(train_ft, target, SEED)
lgb_study.optimize(lgb_objective_func, n_trials=100)

[I 2024-11-13 14:46:16,869] A new study created in memory with name: no-name-45de249e-62a9-40fe-9be1-22d738157151
[I 2024-11-13 14:46:41,209] Trial 0 finished with value: 0.6939550895637453 and parameters: {'n_estimators': 83, 'learning_rate': 0.31, 'max_depth': 13, 'num_leaves': 36, 'min_child_samples': 5}. Best is trial 0 with value: 0.6939550895637453.
[I 2024-11-13 14:49:01,253] Trial 1 finished with value: 0.7066726359180711 and parameters: {'n_estimators': 418, 'learning_rate': 0.22, 'max_depth': 13, 'num_leaves': 83, 'min_child_samples': 6}. Best is trial 1 with value: 0.7066726359180711.
[I 2024-11-13 14:50:10,268] Trial 2 finished with value: 0.7143691956826744 and parameters: {'n_estimators': 212, 'learning_rate': 0.04, 'max_depth': 15, 'num_leaves': 71, 'min_child_samples': 13}. Best is trial 2 with value: 0.7143691956826744.
[I 2024-11-13 14:50:29,823] Trial 3 finished with value: 0.7104700141610285 and parameters: {'n_estimators': 78, 'learning_rate': 0.09999999999999999, 

In [64]:
lgb_study.best_value, lgb_study.best_trial.params

(0.7176274297305094,
 {'n_estimators': 289,
  'learning_rate': 0.04,
  'max_depth': 15,
  'num_leaves': 62,
  'min_child_samples': 23})

In [65]:
# (0.8977595290050033,
#  {'n_estimators': 422,
#   'learning_rate': 0.03,
#   'max_depth': 12,
#   'num_leaves': 50,
#   'min_child_samples': 30,
#   'subsample': 0.8,
#   'colsample_bytree': 0.5,
#   'reg_alpha': 0.8,
#   'reg_lambda': 1.0})

# cv 점수 확인하기

In [66]:
cv = KFold(5, random_state=SEED, shuffle=True)

In [67]:
models = [
    #('lr', LogisticRegression(random_state=SEED, **lr_study.best_trial.params)),
    #('rf', RandomForestClassifier(random_state=SEED, **rf_study.best_trial.params)),
    ('xg', XGBClassifier(random_state=SEED, **xg_study.best_trial.params)),
    ('lgb', LGBMClassifier(random_state=SEED, **lgb_study.best_trial.params))
    ]

NameError: name 'lr_study' is not defined

In [ ]:
model_score = []

for model_name, model in models:
    scores = cross_val_score(model, train_ft, target, cv=cv, n_jobs=-1, scoring="f1_macro")
    model_score.append((model_name, scores.mean()))

In [ ]:
model_score

In [ ]:
model_score

# 모델 학습및 테스트 데이터 예측

In [ ]:
# from xgboost import XGBClassifier

# model = XGBClassifier(
#     random_state=SEED,
#     n_estimators=193,
#     learning_rate=0.15,
#     max_depth=3,
#     gamma=2.6,
#     subsample=0.7,
#     colsample_bytree=0.6
# )


- xg

In [ ]:
model_xg = XGBClassifier(random_state=SEED, **xg_study.best_trial.params)

In [ ]:
model_xg.fit(train_ft, target)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.9, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=2.5000000000000004, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.06999999999999999,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=196, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [ ]:
pred_proba_xg = model_xg.predict_proba(test_ft)[:,1]
pred_proba_xg.shape

(12225,)

In [ ]:
pred_xg = model_xg.predict(test_ft)
pred_xg

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
test_ft.shape

(12225, 494)

In [ ]:
pred_xg_train = model_xg.predict(train_ft)
pred_xg_train

array([0, 1, 0, ..., 0, 1, 1])

In [ ]:
from sklearn.metrics import f1_score
f1_score(target, pred_xg_train, average='macro')

0.8242195210653409

- lgb

In [ ]:
model_lgb = LGBMClassifier(random_state=SEED, **lgb_study.best_trial.params)

In [ ]:
model_lgb.fit(train_ft, target)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 5874, number of negative: 9066
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.072817 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 54952
[LightGBM] [Info] Number of data points in the train set: 14940, number of used features: 470
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.393173 -> initscore=-0.433995
[LightGBM] [Info] Start training from score -0.433995
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


LGBMClassifier(learning_rate=0.03, max_depth=14, min_child_samples=24,
               n_estimators=278, num_leaves=40, random_state=42)

In [ ]:
pred_proba_lgb = model_lgb.predict_proba(test_ft)[:,1]
pred_proba_lgb.shape

(12225,)

In [ ]:
pred_lgb = model_lgb.predict(test_ft)
pred_lgb

array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:
test_ft.shape

(12225, 494)

In [ ]:
pred_lgb_train = model_lgb.predict(train_ft)
pred_lgb_train

array([0., 1., 0., ..., 0., 1., 1.])

In [ ]:
from sklearn.metrics import f1_score
f1_score(target, pred_lgb_train, average='macro')

0.8578391251839137

In [ ]:
#앙상블
pred = pred_proba_xg * 0.5 + pred_proba_lgb * 0.5
pred

array([0.10219475, 0.43720395, 0.326997  , ..., 0.06162459, 0.19311939,
       0.0222828 ])

In [ ]:
pred_final = (pred > 0.5).astype(int)
pred_final.sum()

4089

# 예측 파일 저장후 컴피티션 페이지에 제출

In [ ]:
submit

,ID,target
0,test_0,0.5
1,test_1,0.5
2,test_2,0.5
3,test_3,0.5
4,test_4,0.5
...,...,...
12220,test_12220,0.5
12221,test_12221,0.5
12222,test_12222,0.5
12223,test_12223,0.5


In [ ]:
submit["target"] = pred
submit

,ID,target
0,test_0,0.102195
1,test_1,0.437204
2,test_2,0.326997
3,test_3,0.821938
4,test_4,0.655256
...,...,...
12220,test_12220,0.778495
12221,test_12221,0.298809
12222,test_12222,0.061625
12223,test_12223,0.193119


#업로드

In [ ]:
submit.to_csv(f"{DATA_PATH}submit_새출발_튜닝_2_proba.csv",index=False)